# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [1]:
## Task1
import cv2
import open3d as o3d
from matplotlib import pyplot as plt
import numpy as np
import copy
import scipy
from scipy import spatial
import random
import sys
import math

def compute_transformation(source, target):
    # Kabsch Algorithm
    # Normalization
    number = len(source)
    cs = np.zeros((3,1)) # the centroid of source points
    ct = copy.deepcopy(cs) # the centroid of the target points
    cs[0]=np.mean(source[:][0]);cs[1]=np.mean(source[:][1]);cs[2]=np.mean(source[:][2])
    ct[0]=np.mean(target[:][0]);ct[1]=np.mean(target[:][1]);ct[2]=np.mean(target[:][2])
    cov = np.zeros((3,3)) # covariance matrix
    for i in range(number):
        sources = source[i].reshape(-1,1)-cs
        targets = target[i].reshape(-1,1)-ct
        cov = cov + np.dot(sources,np.transpose(targets))
    # SVD
    u,w,v = np.linalg.svd(cov)
    # rotation matrix
    det = np.linalg.det(cov)
    sign = det/abs(det)
    tran = np.array([[1,0,0],[0,1,0],[0,0,sign]])
    R = np.dot(np.dot(v, tran,),np.transpose(u))
    # Translation Vector
    T = ct - np.dot(R, cs)
    return R, T

def _transform(source, R, T):
    points = []
    for point in source:
        points.append(np.dot(R, point.reshape(-1,1))+T)
    return points

def compute_rmse(source, target, R, T):
    rmse = 0
    number = len(target)
    points = _transform(source, R, T)
    for i in range(number):
        error = target[i].reshape(-1,1)-points[i]
        rmse = rmse + math.sqrt(error[0]**2+error[1]**2+error[2]**2)
    return rmse


def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor): # recolor the points
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None): # transforma source to targets
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

#draw_registrations(source, target)
def pc2array(pointcloud):
    return np.asarray(pointcloud.points)
def registration_RANSAC(source, target, source_feature, target_feature, ransac_n = 3, max_interation=100000, max_validation=100):
    # the intention of RANSAC is to get the optimal transformation between the source and target
    s = pc2array(source) #(4760, 3)
    t = pc2array(target)
    sf = np.transpose(source_feature.data) # (33,4760), source features
    tf = np.transpose(target_feature.data) # target features
    tree = spatial.KDTree(tf) # Create a KD tree 
    corres_stock = tree.query(sf)[1]
    for i in range(max_interation):
        # take ransac_n points randomly
        idx = [random.randint(0, s.shape[0]-1) for j in range(ransac_n)]
        corres_idx = corres_stock[idx]
        source_point = s[idx,...]
        target_point = t[corres_idx, ...]
        # estimate transformation
        # use Kabsch Algorithm
        R, T = compute_transformation(source_point, target_point)
        # calculate rmse for all the points
        source_point = s
        target_point = t[corres_stock,...]
        rmse = compute_rmse(source_point, target_point, R, T)
        if not i:
            opt_rmse = rmse
            opt_R = R
            opt_T = T
        else:
            if rmse<opt_rmse:
                opt_rmse = rmse
                opt_R = R
                opt_T = T
    return opt_R, opt_T
# Used for downsampling.
voxel_size = 0.05
def get_fpfh(cp):
    cp = cp.voxel_down_sample(voxel_size)
    cp.estimate_normals()
    return cp, o3d.registration.compute_fpfh_feature(cp, o3d.geometry.KDTreeSearchParamHybrid(radius=5, max_nn=100))
source = o3d.io.read_point_cloud('global_registration/r1.pcd')
target = o3d.io.read_point_cloud('global_registration/r2.pcd')  
#source = o3d.io.read_point_cloud('global_registration/car1.ply')
#target = o3d.io.read_point_cloud('global_registration/car2.ply') #效果不好应该只是力度不够大               
r1, f1 = get_fpfh(source)
r2, f2 = get_fpfh(target)
#draw_registrations(r1, r2, None, True)
R, T = registration_RANSAC(r1,r2,f1,f2)
transformation = np.vstack((np.hstack((np.float64(R), np.float64(T))), np.array([0,0,0,1])))
draw_registrations(r1, r2, transformation, True)

In [2]:
point2point =  o3d.registration.TransformationEstimationPointToPoint(False)
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    r1, r2, 
    f1, f2, 
    voxel_size*1.5, 
point2point)
draw_registrations(r1, r2, ransac_result.transformation, True)

In [73]:
# Chanllenge的思路（计算量太大，就不做了）
## local registration
# 对点云中的每个点，找到离它最近的点（欧氏距离）->用Kabcsh方法计算transformation->transform points in point cloud->迭代直到精度满足要求
## reconstruct car
#由于数据是深度相机得到的，首先生成深度图->深度图生成点云->点云进行global registration和local registration，第n+1张为source，第n张为target，非常大的计算量，最后可以重建出整车。
